In [ ]:
# sample data is ready - Done 
# code to read the sample data and extracted chemicals -  done  
# code to convert raw md file to readble md file - done
# code to chunk the md file - chunk size 2000 tokens - done
# store the chemicals to flashtext - to be done 
# chemicals in the chunk - chunk wise chemicals for prompt buulding - done
# prompt input instruction 
# value_extraction_prompt - to be done 
# complete prompt building - to be done
# loop over chunks - other value extraction - to be done
    ## other value mapping to RR chemicals - to be done 
# evaluation scripts to evaluate columns wise - to be done
# test on multipule files - to be done

In [ ]:
# imports 
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import re
from markdownify import markdownify as md
from bs4 import BeautifulSoup

In [ ]:
import logging
import traceback
import json
import pandas as pd

In [ ]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [ ]:

@staticmethod
def remove_html_comments(md_content):
    # This regex removes everything from <!-- to -->
    return re.sub(r'<!--.*?-->', '', md_content, flags=re.DOTALL)

@staticmethod
def convert_html_tables_to_markdowns(md_content):
    # Remove HTML comments
    md_content = remove_html_comments(md_content)

    new_md_content = md_content
    try:
        soup = BeautifulSoup(md_content, "html.parser")
        for table in soup.find_all("table"):
            html_str = str(table)
            markdown_table = md(html_str).strip()

            if html_str in new_md_content:
                new_md_content = new_md_content.replace(html_str, markdown_table)
            else:
                start_index = new_md_content.find("<table")
                end_index = new_md_content.find("</table>", start_index) + len("</table>")
                if start_index != -1 and end_index != -1:
                    new_md_content = (
                        new_md_content[:start_index]
                        + markdown_table
                        + new_md_content[end_index:]
                    )
    except Exception as e:
        logger.error(f"Error converting HTML tables to markdown: {str(e)}")
        logger.error(traceback.format_exc())

    return new_md_content.lstrip().rstrip()


In [ ]:
def creat_chunks(md_path):
    # load the file and chunking it 

    loader = TextLoader(md_path,encoding='utf-8')

    md_content = loader.load()
    
    md_content = md_content[0].page_content

    md_content = convert_html_tables_to_markdowns(md_content)

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

    chunks = text_splitter.split_text(md_content)

    return chunks

In [ ]:
md_path = "data\\US_DEA_Source_file.md"
chunks = creat_chunks(md_path)

In [ ]:
# code to read extracted chemicals
import pandas as pd

df = pd.read_excel("data\\US_DEA_chemicals.xlsx", header=0)

df.columns = df.iloc[0]

In [ ]:
df.columns = df.iloc[0]

In [ ]:
extracted_chemicals = df['Chemical Name'].values.tolist()[1:]

In [ ]:
# flashtext chemical annotation 
from flashtext import KeywordProcessor
keyword_processor = KeywordProcessor(case_sensitive=False)

for chemical in extracted_chemicals:
    keyword_processor.add_keyword( chemical )




In [ ]:
# chunk wise available chemicals retrival
chemical_in_chunk = keyword_processor.extract_keywords(chunks[18])

In [ ]:
print(chunks[18])

In [ ]:
context_example = """
                            | --- | --- |
                            | (19) Pethidine (meperidine) | 9230 |
                            | (20) Pethidine-Intermediate-A, 4-cyano-1-methyl-4-phenylpiperidine | 9232 |
                            | (21) Pethidine-Intermediate-B, ethyl-4-phenylpiperidine-4-carboxylate | 9233 |
                            | (22) Pethidine-Intermediate-C, 1-methyl-4-phenylpiperidine-4-carboxylic acid | 9234 |
                            | (23) Phenazocine | 9715 |
                            | (24) Piminodine | 9730 |
                            | (25) Racemethorphan | 9732 |
                            | (26) Racemorphan | 9733 |
                            | (27) Remifentanil | 9739 |
                            | (28) Sufentanil | 9740 |
                            | (29) Tapentadol | 9780 |
                            | (30) Thiafentanil | 9729 |
                            
                """

value_format_prompt = """ 
                            [
                                {
                                    "Chemical Name": "Pethidine",
                                    "Type": "DEA Number",
                                    "Value": "9230",
                                    "remark" : "present"
                                },
                                {
                                    "Chemical Name": "Tapentadol",
                                    "Type": "DEA Number",
                                    "Value": "9780",
                                    "remark" : "Present"
                                },
                                {
                                    "Chemical Name": "Phenazocine",
                                    "Type": "DEA Number",
                                    "Value": "9715",
                                    "remark" : "Present"
                                },
                            ]
                            """

    # Example JSON format with corrected syntax
value_extract_example = f"""Output format (in JSON): Please format the output strictly as valid JSON, using double quotes (") around all keys and string values.
                            {value_format_prompt}
    
                        """

# Combined value extraction section
value_extraction = f"""Context: {context_example}
Values to extract: {value_extract_example}
"""

# Final prompt template
prompt_template = f"""You are a Chemical relational values extraction expert.

    Instructions for Extracting Other Values:

    You will be provided with list of chemicals and a context, from the given context, extract other values for every chemical given in the list. 
    
    Take the example given below as a refference. If value not exit then return None for that variable

    {value_extraction}

    """ 

In [ ]:
print(prompt_template)

In [ ]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

In [ ]:
model_name = "gpt-4o-mini"
llm_deployment = "gpt-4o-mini"
api_version = "2024-12-01-preview"
embed_deployment = "text-embedding-3-small"
load_dotenv()

llm_client = AzureOpenAI(
                        api_version=api_version,
                        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                        api_key=os.getenv("OPENAI_API_KEY"),
                        azure_deployment = llm_deployment,
                        )


In [ ]:
chunk_text = chunks[18]
chunk_chemical = chemical_in_chunk
input_text = f""" 
list of chemical for which values to be extracted:
{chunk_chemical}

chunk text from which values to be extracted
{chunk_text}

"""

In [ ]:
print(input_text) 

In [ ]:
response = llm_client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": prompt_template,
        },
        {
            "role": "user",
            "content": input_text,
        }
    ],
    max_tokens=4096,
    temperature=0.1,
    top_p=1.0,
    model=llm_deployment
)

In [ ]:
response_data = response.choices[0].message.content

In [ ]:
print(response_data)

In [ ]:
def other_value_extractions_from_llm(prompt_template, input_text):

        response = llm_client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": prompt_template,
            },
            {
                "role": "user",
                "content": input_text,
            }
        ],
        max_tokens=4096,
        temperature=0.1,
        top_p=1.0,
        model=llm_deployment
                            )
        
        response_data = response.choices[0].message.content

        cleaned_response = response_data.replace("```json", "").replace("```", "").lstrip().rstrip()
        # Extra safety: Replace single quotes with double quotes (if clearly JSON-like)

        if cleaned_response.startswith("[{") and "'" in cleaned_response and '"' not in cleaned_response:
            cleaned_response = cleaned_response.replace("'", '"')
            
        chemicals_extracted = json.loads(cleaned_response)

        return chemicals_extracted

In [ ]:
def build_input_text(chunk_chemical,chunk_text):

    input_text = f""" 
    list of chemical for which values to be extracted:
    {chunk_chemical}

    chunk text from which values to be extracted
    {chunk_text}
    """
    return input_text

In [ ]:
chemical_other_values = []

for chunk in chunks:
    
    chemicals_in_chunk = keyword_processor.extract_keywords(chunk)

    if chemicals_in_chunk:

        input_text =  build_input_text(chemicals_in_chunk, chunk)
    
        other_values = other_value_extractions_from_llm(prompt_template, input_text)

        chemical_other_values.extend(other_values)
    
df = pd.DataFrame(chemical_other_values)



In [ ]:
df1 = pd.DataFrame(chemical_other_values)

# clean code

In [1]:
import pandas as pd
from pre_processing import Pre_processing
from prompt_building import prompt_building
import re
from other_value_evaluation import ValuesComparision

# === Config === #
md_path = "data\\A_SUSMP\\A_SUSMP_S5.md"

extracted_chemical_path = "data\\A_SUSMP\\A_susmp_2215_extracted_chemicals.xlsx"

ground_truth_path = "data\\A_SUSMP\\A_susmp_2215_gt.xlsx"

compare_columns = ["value", "unit", "type", "remark", "listedunder"]

c:\Users\FL_LPT-706\Documents\Other_value_extraction_pipeline\ot_pipeline3_12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-22 21:06:38,619 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cpu
2025-06-22 21:06:38,620 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


In [2]:
# Inputs from user/UI
context_example = ""  # Example context for value format
value_format_prompt = [{}]  # List of JSON examples for value format

In [3]:
chunks = Pre_processing.creat_chunks(md_path, chunk_size=2000)
print(f"chunks created total chunks {len(chunks)}")

chunks created total chunks 33


In [4]:
print(chunks[0])

# Schedule 5-Caution

Note:
See paragraph (e) of the definition of designated solvent in section 6, paragraph 7(j), sections 16 and 25,
subsection 26(2) and sections 42, 46, 47, 49, 55 and 61.


## ABAMECTIN

(a) in preparations, for internal use for the treatment of animals, containing 1% or less
of abamectin; or

(b) in gel formulations containing 0.05% or less of abamectin in applicators
containing 50 mg or less of abamectin.

ABSCISIC ACID.

ACEQUINOCYL.

ACETIC ACID (excluding its salts and derivatives) in preparations containing more than 30%
of acetic acid (CH3COOH) except:

(a) when included in Schedule 2 or 6; or

(b) for therapeutic use.

ACETONE except in preparations containing 25% or less of designated solvents.

ACRIFLAVINIUM CHLORIDE in preparations for veterinary use containing 2.5% or less of
acriflavinium chloride.

AFOXOLANER in oral divided preparations each containing 150 mg or less of afoxolaner
per dosage unit

(a) for the treatment and prevention of flea infesta

In [5]:
df = pd.read_excel(extracted_chemical_path, header=0)

In [6]:
df

,CAS,Chemical Name
0,50-28-2,Estradiol
1,50-31-7,"2,3,6-TBA"
2,50-78-2,Aspirin
3,55-38-9,Fenthion
4,55-38-9,Fenthion
...,...,...
6707,RR-198943-1,Tigolaner salts of derivatives
6708,RR-198944-2,Tigolaner salts of esters
6709,RR-198945-3,Tigolaner salts of ethers
6710,RR-198946-4,Tigolaner salts of stereoisomers


In [7]:
keyword_processor = Pre_processing.add_extracted_chemicas_to_flashtext(extracted_chemical_path)
print(f"added extrcated chemicals to keyword processor")

added extrcated chemicals to keyword processor


In [8]:
prompt_template = prompt_building.build_prompt(context_example, value_format_prompt)

In [9]:
print(prompt_template)

You are a Chemical relational values extraction expert.

            Instructions for Extracting Other Values:

            You will be provided with list of chemicals and a context, from the given context, extract other values for every chemical given in the list. 
            
            Take the example given below as a refference. If value not exit then return None for that variable

            Context: 
                        PHENOL, including cresols and xylenols and any other homologue of phenol boiling below
                        220°C, when in animal feed additives containing 15% or less of such substances,
                        except in preparations containing 1% or less of phenol and in preparations containing
                        3% or less of cresols and xylenols and other homologues of phenol.

                        PHENYL METHYL KETONE except in preparations containing 25% or less of designated
                        solvents

                        PERMET

In [ ]:
chemicals_in_chunk = keyword_processor.extract_keywords(chunks[0])

In [10]:
chemical_other_values = []

for i, chunk in enumerate(chunks):
    
    chemicals_in_chunk = keyword_processor.extract_keywords(chunk)

    chemicals_in_chunk = list(set(chemicals_in_chunk))

    if not chemicals_in_chunk:
        print(f"#### NO values extracted for the chunk number {i}, number of chemicals in the chunk 0")
        continue

    input_text = prompt_building.build_input_text(chemicals_in_chunk, chunk)

    extracted_values = prompt_building.other_value_extractions_from_llm(prompt_template, input_text)
    
    print(f">>>> chunk number {i} >>>>extracted chemicals {len(extracted_values)}>>>>actual chemicals {len(chemicals_in_chunk)} ")
    print(f"################ >>>>>>> missed chemicals {len(chemicals_in_chunk)-len(extracted_values)}>>>>>>>>################ ")

    chemical_other_values.extend(extracted_values)


2025-06-22 21:09:05,722 - httpx - INFO - HTTP Request: POST https://ulaihub4315577908.cognitiveservices.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


>>>> chunk number 0 >>>>extracted chemicals 11>>>>actual chemicals 11 
################ >>>>>>> missed chemicals 0>>>>>>>>################ 


2025-06-22 21:09:12,931 - httpx - INFO - HTTP Request: POST https://ulaihub4315577908.cognitiveservices.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


>>>> chunk number 1 >>>>extracted chemicals 17>>>>actual chemicals 17 
################ >>>>>>> missed chemicals 0>>>>>>>>################ 


2025-06-22 21:09:19,883 - httpx - INFO - HTTP Request: POST https://ulaihub4315577908.cognitiveservices.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


...

>>>> chunk number 30 >>>>extracted chemicals 22>>>>actual chemicals 22 
################ >>>>>>> missed chemicals 0>>>>>>>>################ 


2025-06-22 21:13:05,789 - httpx - INFO - HTTP Request: POST https://ulaihub4315577908.cognitiveservices.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


>>>> chunk number 31 >>>>extracted chemicals 18>>>>actual chemicals 18 
################ >>>>>>> missed chemicals 0>>>>>>>>################ 


2025-06-22 21:13:12,538 - httpx - INFO - HTTP Request: POST https://ulaihub4315577908.cognitiveservices.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


>>>> chunk number 32 >>>>extracted chemicals 16>>>>actual chemicals 16 
################ >>>>>>> missed chemicals 0>>>>>>>>################ 


In [11]:
chemical_other_values

[{'Chemical Name': 'ALBENDAZOLE',
  'value': '<=12.5',
  'unit': '%',
  'type': 'None',
  'remark': 'for the treatment of animals, except in intraruminal implants each containing 3.85 g or less of albendazole.',
  'listedunder': 'None'},
 {'Chemical Name': 'MILBEMYcin OXIME',
  'value': 'None',
  'unit': 'None',
  'type': 'None',
  'remark': 'when combined with afoxolaner for the treatment and prevention of flea infestations, control of ticks, gastrointestinal nematodes and heartworm in dogs.',
  'listedunder': 'None'},
 {'Chemical Name': 'AKLOMIDE',
  'value': 'None',
  'unit': 'None',
  'type': 'None',
  'remark': 'None',
  'listedunder': 'None'},
 {'Chemical Name': 'ACETONE',
  'value': '<=25',
  'unit': '%',
  'type': 'None',
  'remark': 'except in preparations containing 25% or less of designated solvents.',
  'listedunder': 'None'},
 {'Chemical Name': 'ABAMECTIN',
  'value': '<=1',
  'unit': '%',
  'type': 'None',
  'remark': 'in preparations, for internal use for the treatment o

In [12]:
df = pd.DataFrame(chemical_other_values)
# df.to_csv(extracted_csv_path, index=False)

In [13]:
df

,Chemical Name,value,unit,type,remark,listedunder
0,ALBENDAZOLE,<=12.5,%,None,"for the treatment of animals, except in intrar...",None
1,MILBEMYcin OXIME,None,None,None,when combined with afoxolaner for the treatmen...,None
2,AKLOMIDE,None,None,None,None,None
3,ACETONE,<=25,%,None,except in preparations containing 25% or less ...,None
4,ABAMECTIN,<=1,%,None,"in preparations, for internal use for the trea...",None
...,...,...,...,...,...,...
527,Triisopropanolamine lauryl ether sulfate,<=30,%,None,when labelled with the statements: (a) Avoid c...,None
528,Triflumuron,None,None,None,None,None
529,Trietazine,None,None,None,None,None
530,Trolamine,<=5,%,None,excluding its salts and derivatives.,None


In [14]:
df.to_excel("data\\A_SUSMP\\predicted_othervalues.xlsx",index=False)

In [15]:
# merge the other values extracted with extracted chemicals
df1 = pd.read_excel(extracted_chemical_path,header=0)

# df1.columns = df1.iloc[0]  # First row becomes header
# df1 = df1[1:]              # Drop the first row from data
# df1 = df1.reset_index(drop=True)

In [16]:
df1

,CAS,Chemical Name
0,50-28-2,Estradiol
1,50-31-7,"2,3,6-TBA"
2,50-78-2,Aspirin
3,55-38-9,Fenthion
4,55-38-9,Fenthion
...,...,...
6707,RR-198943-1,Tigolaner salts of derivatives
6708,RR-198944-2,Tigolaner salts of esters
6709,RR-198945-3,Tigolaner salts of ethers
6710,RR-198946-4,Tigolaner salts of stereoisomers


In [17]:
merged_df = pd.merge(df1, df, on='Chemical Name', how='left')

In [18]:
merged_df

,CAS,Chemical Name,value,unit,type,remark,listedunder
0,50-28-2,Estradiol,None,None,None,in implant preparations for growth promotion i...,None
1,50-31-7,"2,3,6-TBA",None,None,None,None,None
2,50-31-7,"2,3,6-TBA",None,None,None,None,None
3,50-78-2,Aspirin,NaN,NaN,NaN,NaN,NaN
4,55-38-9,Fenthion,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
6769,RR-198943-1,Tigolaner salts of derivatives,NaN,NaN,NaN,NaN,NaN
6770,RR-198944-2,Tigolaner salts of esters,NaN,NaN,NaN,NaN,NaN
6771,RR-198945-3,Tigolaner salts of ethers,NaN,NaN,NaN,NaN,NaN
6772,RR-198946-4,Tigolaner salts of stereoisomers,NaN,NaN,NaN,NaN,NaN


In [19]:
merged_df.to_excel("data\\A_SUSMP\\merged_base_othervalues.xlsx",index=False)

In [20]:
def map_othervalues_to_RR_chemicals(df):
    # Load input
    # df = pd.read_csv("inputrr1.csv")

    # Ensure consistent column order
    columns_to_copy = [col for col in df.columns if col not in ["CAS", "Chemical Name"]]

    # Split base and RR entries
    base_df = df[~df["CAS"].str.startswith("RR-")].copy()
    rr_df = df[df["CAS"].str.startswith("RR-")].copy()

    # Normalize names
    def normalize(name):
        return str(name).lower().strip()

    # Fill RR rows with corresponding base info using strict word-boundary matching
    for rr_idx, rr_row in rr_df.iterrows():
        rr_name = normalize(rr_row["Chemical Name"])
        
        for _, base_row in base_df.iterrows():
            base_name = normalize(base_row["Chemical Name"])
            
            # Use \b to ensure it matches whole words or with space boundaries
            if re.search(rf"\b{re.escape(base_name)}\b", rr_name):
                for col in columns_to_copy:
                    rr_df.at[rr_idx, col] = base_row.get(col, "")
                break  # Use first matching base chemical

    # Combine base + updated RR rows back
    final_df = pd.concat([base_df, rr_df], ignore_index=True)

    # # Save output
    # final_df.to_csv("output_with_base_data1.csv", index=False)
    # print(final_df)

    return final_df


In [21]:
mapped_df = map_othervalues_to_RR_chemicals(merged_df)

In [22]:
mapped_df 

,CAS,Chemical Name,value,unit,type,remark,listedunder
0,50-28-2,Estradiol,None,None,None,in implant preparations for growth promotion i...,None
1,50-31-7,"2,3,6-TBA",None,None,None,None,None
2,50-31-7,"2,3,6-TBA",None,None,None,None,None
3,50-78-2,Aspirin,NaN,NaN,NaN,NaN,NaN
4,55-38-9,Fenthion,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
6769,RR-198943-1,Tigolaner salts of derivatives,<=10,%,None,in preparations containing 10% or less of tigo...,None
6770,RR-198944-2,Tigolaner salts of esters,<=10,%,None,in preparations containing 10% or less of tigo...,None
6771,RR-198945-3,Tigolaner salts of ethers,<=10,%,None,in preparations containing 10% or less of tigo...,None
6772,RR-198946-4,Tigolaner salts of stereoisomers,<=10,%,None,in preparations containing 10% or less of tigo...,None


In [23]:
mapped_df.to_excel("data\\A_SUSMP\\rr_mapped_other_values.xlsx", index=False)

In [24]:
# pd.read_excel("data\\rr_mapped_other_values.xlsx")

In [25]:
ground_truth_df = pd.read_excel(ground_truth_path)
ground_truth_df

,CAS,Chemical Name,value,unit,type,remark,listedunder,Formula
0,50-28-2,Estradiol,NaN,NaN,NaN,in implant preparations for growth promotion i...,NaN,C18H24O2
1,50-31-7,"2,3,6-TBA",NaN,NaN,NaN,present,NaN,C7H3Cl3O2
2,50-78-2,Aspirin,NaN,NaN,NaN,for the treatment of animals; that is in divid...,NaN,C9H8O4
3,55-38-9,Fenthion,<=25,%,NaN,in preparations when packed in single use cont...,NaN,C10H15O3PS2
4,55-38-9,Fenthion,<=10,%,NaN,in preparations,NaN,C10H15O3PS2
...,...,...,...,...,...,...,...,...
6707,RR-198943-1,Tigolaner salts of derivatives,<=10,%,NaN,in preparations,NaN,NaN
6708,RR-198944-2,Tigolaner salts of esters,<=10,%,NaN,in preparations,NaN,NaN
6709,RR-198945-3,Tigolaner salts of ethers,<=10,%,NaN,in preparations,NaN,NaN
6710,RR-198946-4,Tigolaner salts of stereoisomers,<=10,%,NaN,in preparations,NaN,NaN


In [26]:

evaluation_results = ValuesComparision.evaluate_extraction_accuracy(ground_truth_df, mapped_df, compare_columns)


Batches: 100%|██████████| 1/1 [00:00<00:00, 58.73it/s]


❌ Mismatched records saved to 'mismatched_rows.csv'
✅ All extracted rows found a match.

🔍 Per-column Accuracy (%):
value       : 28.84%
unit        : 29.95%
type        : 0.0%
remark      : 34.6%
listedunder : 0.0%


# Compliled code

In [ ]:
import pandas as pd
from pre_processing import Pre_processing
from prompt_building import prompt_building
import re
from other_value_evaluation import ValuesComparision

# === Config === #
md_path = "data\\US_DEA_Source_file.md"
chemicals_xlsx_path = "data\\US_DEA_chemicals.xlsx"

# Inputs from user/UI
context_example = ""  # Example context for value format
value_format_prompt = [{}]  # List of JSON examples for value format

chunks = Pre_processing.creat_chunks(md_path, chunk_size=2000)
print(f"chunks created total chunks {len(chunks)}")

keyword_processor = Pre_processing.add_extracted_chemicas_to_flashtext(chemicals_xlsx_path)
print(f"added extrcated chemicals to keyword processor")

prompt_template = prompt_building.build_prompt(context_example, value_format_prompt)

chemical_other_values = []

for i, chunk in enumerate(chunks):
    chemicals_in_chunk = keyword_processor.extract_keywords(chunk)

    if not chemicals_in_chunk:
        print(f"#### NO values extracted for the chunk number {i}, number of chemicals in the chunk 0")
        continue

    input_text = prompt_building.build_input_text(chemicals_in_chunk, chunk)

    extracted_values = prompt_building.other_value_extractions_from_llm(prompt_template, input_text)
    
    print(f">>>> chunk number {i} >>>>extracted chemicals {len(extracted_values)}>>>>actual chemicals {len(chemicals_in_chunk)} ")
    print(f"################ >>>>>>> missed chemicals {len(chemicals_in_chunk)-len(extracted_values)}>>>>>>>>################ ")

    chemical_other_values.extend(extracted_values)


df = pd.DataFrame(chemical_other_values)

df1 = pd.read_excel("data\\US_DEA_chemicals.xlsx",header=0)

merged_df = pd.merge(df1, df, on='Chemical Name', how='left')

# merged_df.to_excel("data\\merged_base_othervalues.xlsx",index=False)

mapped_df = map_othervalues_to_RR_chemicals(merged_df)

# mapped_df.to_excel("data\\rr_mapped_other_values.xlsx", index=False)

ground_truth_df = pd.read_excel("data\\US_DEA_chemicals_gt.xlsx")

compare_columns = ["value", "type", "remark", "listedunder"]

evaluation_results = ValuesComparision.evaluate_extraction_accuracy(ground_truth_df, mapped_df, compare_columns)
